In [1]:
from datasketch import MinHash, MinHashLSH
import pandas as pd
from math import ceil
from IPython.display import display_html 

In [2]:
data = pd.read_csv('tweets_2022_abril_junio.csv')

#### Funcion para obtener los shingles de cada tweet

In [3]:
def get_shingles(text, k):
    shingles = []
    for i in range(len(text)-k+1):
        shingles.append(text[i:i+k].replace('\n', ' '))
    return tuple(shingles)

### Preprocesamiento de los datos y limpieza de datos

In [4]:
#Filtrar el texto
data = data[data['text'].str.contains('RT') == False]

#Dropear columnas innecesarias
data = data.drop(['favorite_count', 'retweet_count', 'created_at'], axis=1)

#Dropear posibles filas duplicadas
data = data.drop_duplicates()

#Pasar los tweets a shingles
data['shingles'] = data.apply(lambda row: get_shingles(row['text'], 4), axis=1)

#Filtrar el dataframe eliminando las columnas que tengan shingles vacios
data = data[data['shingles'].apply(lambda x: len(x) > 0)]

In [5]:
# df que se usara mas adelante para revisar los tweets de cada usuario
usuarios = data.groupby('screen_name', as_index=False).agg({'id': lambda x: list(x)})

# lista de tuplas que se usara para calcular los minhashes
tweets = list(zip(data.id, data.screen_name, data.shingles))

### Crear los minhashes y el lsh con las clases importadas de datasketch

In [6]:
# Los minhashes calculados se guardaran en un diccionario para despues poder acceder a estos sin problema
minhashes = dict()
for tweet in tweets:
    minhash = MinHash(num_perm=64)
    for shingle in tweet[2]:
        minhash.update(shingle.encode('utf8'))
    minhashes[tweet[0]] = [minhash, tweet[1]]

In [7]:
# Se instancia la clase MinHashLSH con nuestro threshold definido
lsh = MinHashLSH(threshold=0.8, num_perm=64)
for key in minhashes:
    lsh.insert((key, minhashes[key][1]), minhashes[key][0])

#### Definicion de funcion get_count()

In [8]:
def get_count(fila, user):
    n = 0
    for item in fila:
        if user in item:
            n += 1
    return n

#### Crear un diccionario que nos servira para revisar la cantidad de tweets por persona

In [9]:
counts = data.groupby('screen_name')['text'].count()
cant_tweets = counts.to_dict()

### Codigo de comparacion de usuarios
##### En la siguiente celda de codigo se comparan todos los tweets de un usuario con todos sus tweets parecidos resultantes del lsh para despues guardar el usuario al cual pertenece el tweet. Definimos la variable THRESHOLD la cual representa el porcentaje minimo de tweets los cuales tienen que ser similares a tweets de otro usuario para ser considerado como un usuario similar. Por ejemplo si un usuario tiene 100 tweets, otro con 50 tweets y se setea un threshold de 0.4 entonces el primer usuario tiene que tener por lo menos 40 tweets parecidos al segundo usuario y el segundo usuario tiene que tener por lo menos 20 tweets parecidos al primer usuario, si se cumplen estas 2 condiciones entonces se consideran usuarios similares.

In [75]:
# setear porcentaje de tweets minimos para considerar que 2 usuarios son similares
THRESHOLD = 0.35

# A este diccionario y set se le insertaran los output de usuarios similares
usuarios_final = dict() # Tendra 1 usuario como llave y como value tendra una lista de usuarios los cuales se consideran similares
mega_set = set() # Tendra tuplas de pares de usuarios los cuales se consideran similares


for i, row in usuarios.iterrows():
    fila = []
    set_similares = set()
    usuario = row[0]
    ids = row[1]


    # Definir que int es el threshold a superar
    threshold_usuario = ceil(cant_tweets[usuario] * THRESHOLD)
    usuarios_final[usuario] = list()
    set_comparaciones = set()
    uniques_count = dict()

    # Recorrer las ids de los tweets de un usuario
    for id_ in ids:
        temp_set = set()
        result = lsh.query(minhashes[id_][0])
        appaered = []

        # Recorrer las tuplas de posibles tweets parecidos que tienen la forma (id, usuario)
        for tupla in result:
            usuario2 = tupla[1]
            id_usuario2 = tupla[0]

            if usuario2 == usuario:
                continue
            
            if usuario2 not in set_comparaciones:
                set_comparaciones.add(usuario2)
                uniques_count[usuario2] = []

            if usuario2 not in appaered:
                appaered.append(usuario2)
                if id_usuario2 not in uniques_count[usuario2]:
                    uniques_count[usuario2].append(id_usuario2)

            temp_set.add(usuario2)

        fila.append(temp_set)
        set_similares = set_similares.union(temp_set)

    # Revisar los usuarios en el set de posibles similares
    for el in set_similares:

        # Ver cuantas veces aparece en la lista fila y definir un int como threshold para ese usuario
        cuenta = get_count(fila, el)
        tweets_unicos = len(uniques_count[el])
        threshold_comparacion = ceil(cant_tweets[el] * THRESHOLD)

        # Revisar si efectivamente se cumple con el threshold 
        if cuenta >= threshold_usuario and tweets_unicos >= threshold_comparacion:
            
            # Appendear el usuario similar al diccionario y al set
            usuarios_final[usuario].append(el)
            tup = tuple(sorted((usuario, el)))
            mega_set.add(tup)

    # En caso de que el diccionario[key] este vacio se eliminara para no molestar
    if len(usuarios_final[usuario]) == 0:
        del usuarios_final[usuario]

##### Hubieron hartos casos que se salian del molde "normal" al momento de conseguir usuarios parecidos. Nos encontramos con usuarios que se consideran parecidos aunque estos tengan solo un tweet, los cuales dejamos pasar. Tambien encontramos usuarios que tenian 1 tweet parecido a varios tweets distintos de otro usuario, al principio esto nos daba que los usuarios eran parecidos pero lo logramos arreglar implementando sets de usuarios parecidos por cada tweet del usuario principal.

### Se puede pasar la data que tenemos de los pares de usuarios parecidos a un dataframe para visualizarla mejor

In [76]:
mega_set = list(mega_set)
df_pares_parecidos = pd.DataFrame(mega_set, columns=['Usuario1', 'Usuario2'])
df_pares_parecidos

,Usuario1,Usuario2
0,Ezequielabogado,yerko368
1,Oscar270711,fsereyg
2,AntuuneezSeba,victor_sepescud
3,ChaniRayen,mjvrcase
4,hespinosa71,odiliajborje
...,...,...
16781,chacoprensaok,cn38ok
16782,ipakankure,javierledesmare
16783,lasargiento,sheylaarones
16784,PaulaRosso7,RosaElenaLFlore


### Se definen funciones para visualizar y comparar los tweets entre 2 usuarios parecidos

In [77]:
# Funcion que simplemente retorna la lista de usuarios parecidos a un usuario
def encontrar_parecidos(usuario):
    parecidos = usuarios_final[usuario]
    return parecidos

# Funcion que compara dataframes de tweets entre 2 usuarios parecidos
def obtener_tweets_entre_usuarios(usuario1, usuario2):
    tweets_id1 = usuarios.loc[usuarios['screen_name'] == usuario1].values[0][1]
    tweets_id2 = usuarios.loc[usuarios['screen_name'] == usuario2].values[0][1]
    lista1 = []
    lista2 = []
    for id1 in tweets_id1:
        tweet = data.loc[data['id'] == id1].values[0][2]
        lista1.append(tweet)
    for id2 in tweets_id2:
        tweet = data.loc[data['id'] == id2].values[0][2]
        lista2.append(tweet)

    df1 = pd.DataFrame(lista1, columns=[f'Tweets'])
    df2 = pd.DataFrame(lista2, columns=[f'Tweets'])

    df1_styler = df1.style.set_table_attributes("style='display:inline'").set_caption(usuario1)
    df2_styler = df2.style.set_table_attributes("style='display:inline'").set_caption(usuario2)
    display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)
    
    # Se retornan los dataframes por si se quieren trabajar con mas detalle
    return [df1, df2]

### Ejemplos

#### 1.

In [78]:
encontrar_parecidos('CarlosApruebo')

['Elviracristi']

##### Vemos cuales usuarios son parecidos a "CarlosApruebo" y nos da que "Elviracristi" es el unico parecido entonces revisamos sus tweets con la funcion obtener_tweets_entre_usuarios()

In [79]:
dataframes1 = obtener_tweets_entre_usuarios('CarlosApruebo', 'Elviracristi')

,Tweets
0,#AprueboDeSalida #Apruebo4deSeptiembre
1,#AprueboNuevaConstitucion #Apruebo4deSeptiembre
2,#Apruebo4deSeptiembre #AprueboPlebicitoDeSalida
3,@Jaime_Bassa Abrazo @Jaime_Bassa ✊️
4,"No lo hará Aprobaremos, Venceremos y Será Hermoso ✊🏻 #AprueboPlebicitoDeSalida #AprueboNuevaConstitucion"
5,#Apruebo4deSeptiembre #AprueboPlebicitoDeSalida
,Tweets
0,#AprueboDeSalida 🙏🏻
1,#AprueboPlebicitoDeSalida #Apruebo4deSeptiembre
2,#Apruebo4deSeptiembre #Apruebo #NuevaConstitucion


##### Como se puede ver, los 2 usuarios comparten los mismos hashtag en varios de sus tweets lo que lleva a que sean similares al momento de escribir tweets

#### 2.

In [80]:
encontrar_parecidos('Alejand32306857')

['acont77', 'lorenzuela']

##### Aca vemos que "Alejand32306857" tiene 2 usuarios los cuales son parecidos a el, entonces revisamos sus tweets

In [81]:
dataframes2 = obtener_tweets_entre_usuarios('Alejand32306857', 'lorenzuela')

,Tweets
0,"@Jaime_Bassa @RoyoManuela Rechazo, no representa Chile esta constitución, solo favorece comunistas y mapuches narcos"
1,@24HorasTVN @convencioncl @gdominguez_ Rechazo
2,@ElisaLoncon Rechazo
3,@Jaime_Bassa Rechazo
4,@MEQChile Rechazo
,Tweets
0,@MEQChile Rechazo!
1,@gdominguez_ Rechazo!
2,@Jaime_Bassa Rechazo!
3,@MEQChile Qué bueno!! Así tengo más argumentos para rechazar!! Muchas gracias!!


In [82]:
dataframes2 = obtener_tweets_entre_usuarios('Alejand32306857', 'acont77')

,Tweets
0,"@Jaime_Bassa @RoyoManuela Rechazo, no representa Chile esta constitución, solo favorece comunistas y mapuches narcos"
1,@24HorasTVN @convencioncl @gdominguez_ Rechazo
2,@ElisaLoncon Rechazo
3,@Jaime_Bassa Rechazo
4,@MEQChile Rechazo
,Tweets
0,"@patriciapolitz Que deje de leer sus novelas, comedias, historias eroticas y se dedica a cumplir su función de presidente"
1,@PabloSrCasual @danielstingo Estas dando jugo stingo
2,@ipoduje @danielstingo Wueon prepotente
3,@Jaime_Bassa Rechazo!


##### Como podemos ver, los 2 usuarios que se parecen a "Alejand32306857" segun nuestro programa, tweetean las mismas ideas (sobre el rechazo en este caso) y mencionan a las mismas personas.

## Cosas que asumimos

#### 1. Asumimos que los retweets no contaban como tweets originales de un usuario por lo cual no los contamos al momento de comparar tweets, esto nos disminuyo la cantidad de datos considerablemente (alrededor de 1.2 millones).
#### 2. Concluimos que 2 usuarios son parecidos solo si estos comparten una cantidad de tweets similares que son proporcionales a la cantidad de tweets que tiene cada uno, osea una cantidad de tweets que cumplan con el porcentaje minimo.
#### 3. Consideramos que los tweets mas cortos de 4 letras no eran relevantes por lo cual dropeamos esas filas (tambien se debe a que nuestro valor k era 4 y no se pueden sacar shingles de tamaño 4 si el tweet tiene menos de 4 caracteres), esto nos elimino varios tweets de solo emojis.
#### 4. Los usuarios que tenian solo 1 tweet los dejamos para las comparaciones entre usuarios, pensamos filtrarlos debido a la poca información que un solo tweet entrega, pero llegamos a la conclusion de que esto quitaria una cantidad muy considerable de datos.

## Parametros finales
#### k = 4: Esta seria la variable que representa los k-shingles.
#### s = 0.8: Esta seria nuestra variable que representa el threshold usado en la clase MinHashLSH para filtrar posibles parecidos.
#### THRESHOLD = 0.35: Esta es la variable que representa el porcentaje de tweets minimos que se deben tener en comun para poder asumir que 2 usuarios son parecidos.